In [1]:
from ortools.linear_solver import pywraplp
from ortools.linear_solver import pywraplp
import numpy as np
from itertools import combinations
import pandas as pd
import numpy as np

In [2]:
dose_matrix = pd.read_csv('DoseMatrix.csv')

In [3]:
dose_matrix =np.array(dose_matrix)

In [4]:
CTV_index=[167, 168, 169, 170, 171, 172, 173,
      186, 187, 188, 189, 190, 191, 192, 193, 194,
      206, 207, 208, 209, 210, 211, 212, 213, 214, 
           227, 228, 229, 230, 231, 232, 233]

bladder_index=[87, 88, 89, 90, 91, 92, 93, 
         106, 107, 108, 109, 110, 111, 112, 113, 114, 
         126, 127, 128, 129, 130, 131, 132, 133, 134, 
             147, 148, 149, 150, 151, 152, 153]

Rectal_index=[248, 249, 250, 251, 252, 
        267, 268, 269, 270, 271, 272, 273, 
        287, 288, 289, 290, 291, 292, 293, 
              308, 309, 310, 311, 312]

Un_index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
             21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
             41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 
             61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 
             81, 82, 83, 84, 85, 86, 94, 95, 96, 97, 98, 99, 100, 101, 104, 105, 115, 118, 119, 120,
             125, 139, 140, 145, 146, 154, 159, 160, 165, 166, 174, 179, 180, 185, 199, 200, 205, 219, 220, 
             225, 226, 234, 239, 240, 241, 244, 245, 246, 247, 253, 254, 255, 258, 259, 260, 261, 262, 263, 
             264, 265, 266, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 294, 295, 296, 
             297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 313, 314, 315, 316, 317, 318, 319, 320, 
             321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 
             340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 
             359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 
             378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 
             397, 398, 399]

Left_index=[116, 117, 
       135, 136, 137, 138, 
       155, 156, 157, 158, 
       175, 176, 177, 178,
       195, 196, 197, 198, 
       215, 216, 217, 218, 
       235, 236, 237, 238, 
            256, 257]

Right_index=[101, 102, 
        120, 121, 122, 123, 
        140, 141, 142, 143, 
        160, 161, 162, 163, 
        180, 181, 182, 183, 
        200, 201, 202, 203, 
        220, 221, 222, 223, 
             241, 242]

In [5]:
len(CTV_index)+len(bladder_index)+len(Rectal_index)+len(Un_index)+len(Left_index)+len(Right_index)

400

In [6]:
solver = pywraplp.Solver('Final Project',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective=solver.Objective() 
objective.SetMinimization()

# Decision Variables
beamlets = []
for i in range(60):
    beamlets.append(solver.NumVar(0,solver.infinity(), 'beamlets[%d]' % i))
voxels = []    
for i in range(400):
    voxels.append(solver.NumVar(0,solver.infinity(),'voxels[%d]' % i))

    
#Constraints
voxel_constraints = [0 for _ in range(400)]
for i in range(400):
    voxel_constraints[i] = solver.Constraint(0,0)
    for j in range(60):
        voxel_constraints[i].SetCoefficient(beamlets[j],dose_matrix[i,j+1])
    voxel_constraints[i].SetCoefficient(voxels[i], -1)
        
######################################################## CTV#################################################

# every voxel receives a uniform dose of 82.8       ( = 82.8)
z_p = [0 for _ in range(len(CTV_index))]
z_n = [0 for _ in range(len(CTV_index))]

for i in range(len(CTV_index)):
    z_p[i] = solver.NumVar(0, solver.infinity(), "Z plus CTV %d" %i)
    z_n[i] = solver.NumVar(0, solver.infinity(), "Z minus CTV %d" %i)
     
CTV_constraint = [0 for _ in range(len(CTV_index))]
for i in range(len(CTV_index)):
    CTV_constraint[i] = solver.Constraint(82.8,82.8)
    
    for j in range(60):
        CTV_constraint[i].SetCoefficient(beamlets[j],dose_matrix[CTV_index[i],j+1])
    CTV_constraint[i].SetCoefficient(z_p[i],1)
    CTV_constraint[i].SetCoefficient(z_n[i],-1)

CTV_constraint_2 = [[0 for _ in range(len(CTV_index))]for _ in range(len(CTV_index))]
for i in range(len(CTV_index)):
    for j in range(len(CTV_index)):
        CTV_constraint_2[i][j] = solver.Constraint(0, 0.08*82.8)
        CTV_constraint_2[i][j].SetCoefficient(z_p[i],1)
        CTV_constraint_2[i][j].SetCoefficient(z_n[j],1.08)
        
        
    
############################################ bladder ####################################################

# # max dose to a voxel 81 ( < = 81)
z_b_1 = [0 for _ in range(len(bladder_index))]

for i in range(len(bladder_index)):
    z_b_1[i] = solver.NumVar(0, solver.infinity(), "z bladder #1 %d" %i)
    

bladder_constraint_1 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_1[i] = solver.Constraint(-solver.infinity(),81)
    for j in range(60):
        bladder_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_1[i].SetCoefficient(z_b_1[i],-1)


# Average dose should be <= 50     ( <= 50)

z_b_2 = solver.NumVar(0, solver.infinity(), "z bladder #2 %d" %i)
bladder_constraint_2 = solver.Constraint(-solver.infinity(), 50)
bladder_constraint_2.SetCoefficient(z_b_2, -1)

for i in bladder_index:
    bladder_constraint_2.SetCoefficient(voxels[i],1/len(bladder_index))

        
    
# at most 10% of the bladder should receive a dose >65          ( >65)
y = [0 for _ in range(len(bladder_index))]
z_y = solver.IntVar(0,1, "z y constraint bladder")

for i in range(len(bladder_index)):
     y[i] = solver.IntVar(0,1,"y bladder %d" %i)

bladder_constraint_3 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_3[i] = solver.Constraint(-solver.infinity(),65)
    for j in range(60):
        bladder_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_3[i].SetCoefficient(y[i],-100)

bladder_constraint_3b = solver.Constraint(-solver.infinity(),0.1*len(bladder_index))
for j in range(len(bladder_index)):
     bladder_constraint_3b.SetCoefficient(y[j],1)
bladder_constraint_3b.SetCoefficient(z_y, -1)

        
        

        
# ####################### Rectal########################################

#max dose to a voxel: 79.2              ( < = 79.2)

z_r_1 = [0 for _ in range(len(Rectal_index))]

for i in range(len(Rectal_index)):
    z_r_1[i] = solver.NumVar(0, solver.infinity(), "z rectal #1 %d" %i)
    

Rectal_constraint_1 = [0 for _ in range(len(Rectal_index))]
for i in range(len(Rectal_index)):
    Rectal_constraint_1[i] = solver.Constraint(-solver.infinity(),79.2)
    for j in range(60):
        Rectal_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Rectal_index[i],j+1])
    Rectal_constraint_1[i].SetCoefficient(z_r_1[i],-1)

    
    
# #average dose should be <= 40

z_r_2 = solver.NumVar(0, solver.infinity(), "z rectal #2 %d" %i)
Rectal_constraint_2 = solver.Constraint(-solver.infinity(), 40)
Rectal_constraint_2.SetCoefficient(z_r_2, -1.0)

for i in Rectal_index:
    Rectal_constraint_2.SetCoefficient(voxels[i],1/len(Rectal_index))



# ############################## Unspecified ############################

# max dose to a voxel: 72         ( < =72)

z_u_1 = [0 for _ in range(len(Un_index))]

for i in range(len(Un_index)):
    z_u_1[i] = solver.NumVar(0, solver.infinity(), "z Unspecified #1 %d" %i)
    

Un_constraint_1 = [0 for _ in range(len(Un_index))]
for i in range(len(Un_index)):
    Un_constraint_1[i] = solver.Constraint(-solver.infinity(),72)
    for j in range(60):
        Un_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Un_index[i],j+1])
    Un_constraint_1[i].SetCoefficient(z_u_1[i],-1)


#############################Left femur head ##########################

#max dose to a voxel: 50            ( <= 50)
  
z_l_1 = [0 for _ in range(len(Left_index))]

for i in range(len(Left_index)):
    z_l_1[i] = solver.NumVar(0, solver.infinity(), "z left femur #1 %d" %i)
    

left_constraint_1 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    left_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        left_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    left_constraint_1[i].SetCoefficient(z_l_1[i],-1)    
        

        
# at most 10% of the left femur should receive a dose > 40 
y_left = [0 for _ in range(len(Left_index))]
z_y_left = solver.IntVar(0,2, "z y constraint left femur")

for i in range(len(Left_index)):
     y_left[i] = solver.IntVar(0,1,"y left femur %d" %i)

Left_constraint_3 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    Left_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Left_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    Left_constraint_3[i].SetCoefficient(y_left[i],-100)

Left_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Left_index))
for j in range(len(Left_index)):
     Left_constraint_3b.SetCoefficient(y_left[j],1)
Left_constraint_3b.SetCoefficient(z_y_left,-1)


############################# right femur head ##########################


#max dose to a voxel: 50               ( <= 50 )
z_rl_1 = [0 for _ in range(len(Right_index))]

for i in range(len(Right_index)):
    z_rl_1[i] = solver.NumVar(0, solver.infinity(), "z right femur #1 %d" %i)
    

Right_constraint_1 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    Right_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        Right_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    Right_constraint_1[i].SetCoefficient(z_rl_1[i],-1)    
        

        
# at most 10% of the right femur should receive a dose > 40 
y_right = [0 for _ in range(len(Right_index))]
z_y_right = solver.IntVar(0,2, "z y constraint right femur")

for i in range(len(Right_index)):
     y_right[i] = solver.IntVar(0,1,"y right femur %d" %i)

Right_constraint_3 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    
    # <= 40
    Right_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Right_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    # Big M constraint
    Right_constraint_3[i].SetCoefficient(y_right[i],-100)

Right_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Right_index))
for j in range(len(Right_index)):
     Right_constraint_3b.SetCoefficient(y_right[j],1)
Right_constraint_3b.SetCoefficient(z_y_right, -1)
        
        
########################## Objective Function ###########################

# CTV
for i in range(len(z_p)):
    objective.SetCoefficient(z_p[i],20000)
    objective.SetCoefficient(z_n[i],20000)

# Bladders
for i in range(len(z_b_1)):
    objective.SetCoefficient(z_b_1[i],1)

objective.SetCoefficient(z_b_2,1)

# Rectal Solids
for i in range(len(z_r_1)):
    objective.SetCoefficient(z_r_1[i],1000)
    
objective.SetCoefficient(z_r_2,1)

#Unspecified
for i in range(len(z_u_1)):
    objective.SetCoefficient(z_u_1[i],1)

# left femur head
for i in range(len(z_l_1)):
    objective.SetCoefficient(z_l_1[i],1000)

#right femur head
for i in range(len(z_rl_1)):
    objective.SetCoefficient(z_rl_1[i],1)

# bladder, left femur, right femur's at most constraint
objective.SetCoefficient(z_y,1)
objective.SetCoefficient(z_y_left,1)
objective.SetCoefficient(z_y_right,1)

status = solver.Solve()

if status == solver.OPTIMAL:
    print('Problem solved in %f milliseconds' %solver.wall_time())
elif status == solver.FEASIBLE:
    print('Solver claims feasibility but not optimality')
else:
    print('Solver ran to completion but did not find an optimal solution')

print('weighted slack total',objective.Value()) 


#print(bladder_constraint_2.dual_value())

#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

Problem solved in 37851.000000 milliseconds
weighted slack total 1751086.3214159876


In [7]:
print(bladder_constraint_2.dual_value())

0.0


In [8]:
#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

## All Voxels

In [9]:
All_voxels = np.zeros((400)).astype(np.float)
for i in range(400):
    All_voxels[i] = voxels[i].solution_value()
#print(All_voxels.reshape(20,20))
All_voxels = pd.DataFrame(All_voxels.reshape((20,20)), index=[i for i in range(1,21)],
                          columns = [i for i in range(1,21)]).round(2)
All_voxels

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,61.15,47.56,33.95,26.17,25.01,24.21,18.26,9.91,4.44,3.19,5.08,8.62,13.43,19.91,25.30,26.07,24.15,23.48,25.04,28.03
2,62.15,56.33,45.39,36.21,32.29,29.44,22.47,13.49,7.25,5.29,7.14,11.70,18.34,26.53,32.36,32.19,29.03,27.76,29.12,31.31
3,53.26,57.05,53.20,46.65,41.72,36.39,27.37,17.53,11.19,9.35,11.79,17.64,25.66,34.27,39.29,37.73,33.49,31.53,31.91,32.02
4,38.02,48.85,53.72,53.76,51.44,45.16,33.66,22.23,16.22,16.01,20.22,27.05,34.73,42.04,45.55,42.63,37.18,33.94,32.40,29.72
5,22.31,34.94,46.23,54.32,58.16,54.18,41.72,28.77,23.21,25.54,32.16,39.09,44.72,49.58,51.17,46.58,39.46,34.29,30.29,25.03
6,10.63,20.64,33.62,47.72,59.10,60.80,50.80,38.44,33.73,37.69,45.45,51.72,55.10,57.24,56.27,49.35,40.00,32.46,26.02,19.06
7,4.08,10.03,20.73,36.68,54.04,63.27,59.58,51.23,48.06,51.45,57.80,62.91,65.00,65.00,61.23,51.36,39.08,28.78,20.34,12.87
8,1.27,4.12,11.33,25.85,46.18,62.54,67.24,65.00,63.59,65.00,68.35,71.54,72.83,72.14,66.67,53.57,37.34,23.90,14.26,7.51
9,0.37,1.74,6.67,19.48,40.96,62.25,73.55,76.19,76.19,76.19,76.78,77.37,77.76,77.83,72.29,56.28,35.65,19.21,9.18,3.83
10,0.30,1.59,6.58,20.02,42.99,66.04,78.59,82.09,82.81,82.71,81.93,80.85,80.75,81.55,76.19,58.43,35.06,16.86,6.76,2.27


In [10]:
OUTPUT  = 'output/cross_section_p3.csv'
cross_section = All_voxels.to_csv(OUTPUT, index=[i for i in range(1,21)], columns = [i for i in range(1,21)])

## Check Constraint

**Beamlets Check**

In [11]:
# check beamlets value
beamlets_value = []
for i in range(60):
    beamlets_value.append(beamlets[i].solution_value())
print('Beamlets values are ')
print(np.array(beamlets_value))   

Beamlets values are 
[50.43845111  0.          0.          0.          0.          0.
 15.03947086  7.86649076  0.         45.42306822  0.          0.
  0.          0.          0.          0.29861694 21.00289673  0.
 30.92672474  0.          0.          0.          0.         17.72607875
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         19.14968843 25.50748258 29.64211471  0.
  0.          0.          0.          0.         58.4461621   5.01907445
 61.76836803 26.78295081  0.         35.37092775 35.78554921  0.        ]


**CTV check**

In [12]:
#Every voxel receives a uniform dose of 82.8 
CTV_voxel = []
every = 0
for i in CTV_index:
    every = voxels[i].solution_value() 
    CTV_voxel.append(every)
print('CTV voxel dose(Gy)')
print(np.array(CTV_voxel))

CTV voxel dose(Gy)
[76.18780219 76.18780219 76.18780219 76.77714098 77.36663773 77.7551038
 77.83301214 78.58912122 82.09046074 82.81092796 82.71351287 81.92846393
 80.85384888 80.7538158  81.55119154 76.18780219 82.8        82.81092796
 82.51677043 82.8        82.81092796 82.64372452 82.8        82.81092796
 76.50186898 80.27054339 76.36921416 76.88997075 79.90667134 82.50817487
 82.8        80.95075448]


In [13]:
# the distance to 82.8
zp0 = np.array([[x.solution_value() for x in z_p]])
zn0 = np.array([[x.solution_value() for x in z_n]])
print('Positive distance to 82.8 \n',zp0)
print('\n')
print('Negative distance to 82.8 \n',zn0)

Positive distance to 82.8 
 [[ 6.61219781e+00  6.61219781e+00  6.61219781e+00  6.02285902e+00
   5.43336227e+00  5.04489620e+00  4.96698786e+00  4.21087878e+00
   7.09539258e-01 -4.99020130e-13  8.64871315e-02  8.71536071e-01
   1.94615112e+00  2.04618420e+00  1.24880846e+00  6.61219781e+00
  -4.99018803e-13 -1.39988608e-18  2.83229570e-01 -1.39988608e-18
  -4.99018816e-13  1.56275478e-01 -1.95247638e-12 -4.71942795e-19
   6.29813102e+00  2.52945661e+00  6.43078584e+00  5.91002925e+00
   2.89332866e+00  2.91825126e-01 -4.99019979e-13  1.84924552e+00]]


Negative distance to 82.8 
 [[4.62054447e-13 0.00000000e+00 4.62055537e-13 0.00000000e+00
  4.62054459e-13 4.62054459e-13 0.00000000e+00 4.62054460e-13
  0.00000000e+00 1.09279561e-02 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.29619081e-18 1.29614308e-18 0.00000000e+00
  3.08562696e-12 1.09279561e-02 4.62054459e-13 1.29619081e-18
  1.09279561e-02 0.00000000e+00 0.00000000e+00 1.09279561e-02
  0.00000000e+00 0.00000000e+00 0.000000

In [14]:
print('maximum difference of CTV voxels values to 82.8 are ', (max(CTV_voxel)-82.8)/82.8, (82.8- min(CTV_voxel))/82.8)

maximum difference of CTV voxels values to 82.8 are  0.00013198014609482265 0.07985746144222423


In [15]:
print('maximum CTV voxels value is ', max(CTV_voxel))

maximum CTV voxels value is  82.81092795609665


In [16]:
print('minimum CTV voxels value is ', min(CTV_voxel))

minimum CTV voxels value is  76.18780219258383


In [17]:
print('percentage of voxels below 79:', sum((np.array(CTV_voxel) < 79)*1) / len(CTV_voxel))

percentage of voxels below 79: 0.375


In [18]:
print('percentage of voxels Above 86.94:', sum((np.array(CTV_voxel) > 86.94)*1) / len(CTV_voxel))

percentage of voxels Above 86.94: 0.0


**Bladder Check**

In [19]:
# Max dose to a voxel: 81
bladder_voxel = []
every = 0
for i in bladder_index:
    every = round(voxels[i].solution_value(),4)
    bladder_voxel.append(every)
print('Bladder Voxel Doses\n',np.array(bladder_voxel))

Bladder Voxel Doses
 [28.7708 23.2106 25.5366 32.1555 39.0918 44.717  49.5758 50.7951 38.4414
 33.7347 37.6926 45.4473 51.7212 55.0967 57.2351 56.2652 59.5783 51.2274
 48.0582 51.4455 57.7967 62.9131 65.     65.     61.2277 65.     63.5869
 65.     68.3508 71.5364 72.8291 72.144 ]


In [20]:
# if any voxel has more than 81
zb1 = np.array([[x.solution_value() for x in z_b_1]])
print('distances to 81 if voxel value is above 81\n',zb1)

distances to 81 if voxel value is above 81
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


In [21]:
# check if there are voxel with value more than 81
print('are there values more than 81?')
print(max(bladder_voxel) > 81)

print('how many percentage of voxel values are more than 81?')

print(round(sum((np.array(bladder_voxel) >81)*1)/len(bladder_voxel),2)*100,'%')

are there values more than 81?
False
how many percentage of voxel values are more than 81?
0.0 %


In [22]:
# average dose shouldbe <= 50
print('average dose received by bladder voxels are ',sum(bladder_voxel)/len(bladder_voxel))
print('slack variable to bring down the value to below 50: ', z_b_2.solution_value())

average dose received by bladder voxels are  52.193171875
slack variable to bring down the value to below 50:  2.1931753621115346


In [23]:
# at most 10% of the bladder should receive a dose > 65
y_value = np.array([t.solution_value() for t in y])
print('voxel that are larger that 65 are indicated by 1 \n',y_value)

voxel that are larger that 65 are indicated by 1 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1.]


In [24]:
test = np.array(list(map(lambda x: 1 if x > 65.00 else 0,bladder_voxel)))
print(round(sum(test)/len(test),4), ' of the voxels are above 65')

0.125  of the voxels are above 65


In [25]:
print('voxel that are larger that 65 are indicated by 1 \n',test)

voxel that are larger that 65 are indicated by 1 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]


In [26]:
y_value == test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [27]:
print('the number of constraints we have to set loose to be above 65',z_y.solution_value())

the number of constraints we have to set loose to be above 65 1.0


**Rectal Solid Check**

In [28]:
# max dose to a voxel: 79.2
Rectal_voxel = []
every = 0
for i in Rectal_index:
    every = round(voxels[i].solution_value(),8)
    Rectal_voxel.append(every)
print('rectal voxel dose values')
print(np.array(Rectal_voxel))

rectal voxel dose values
[67.56869621 68.25976529 74.24698358 78.74413373 78.56926956 71.55490377
 60.25251736 60.34143422 65.9590885  69.70228509 69.58681232 68.90427419
 68.58469233 56.23523342 53.45223921 54.85493708 56.36899501 58.02414282
 61.78222682 53.21336604 45.38561012 42.12151037 42.62691084 47.2479847 ]


In [29]:
zr1 = np.array([[x.solution_value() for x in z_r_1]])
print('distances to 79.2 if voxel value is above 79.2\n',zr1)

distances to 79.2 if voxel value is above 79.2
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [30]:
# check if there's value larger than 79.2
print('are there values more than 79.2?')
print(max(Rectal_voxel) > 79.2)

print('how many percentage of voxel values are more than 79.2?')

print(round(sum((np.array(Rectal_voxel) >79.2)*1)/len(Rectal_voxel),2)*100,'%')

are there values more than 79.2?
False
how many percentage of voxel values are more than 79.2?
0.0 %


In [31]:
# average dose should be <= 40
print('average dose received by bladder voxels are ',sum(Rectal_voxel)/len(Rectal_voxel))
print('slack variable to bring down the value to below 40: ',z_r_2.solution_value())

average dose received by bladder voxels are  61.399500524166655
slack variable to bring down the value to below 40:  21.399500523801827


**Unspecified**

In [32]:
# Max dose to a voxel: 72
Un_voxel = []
every = 0
for i in Un_index:
    every = round(voxels[i].solution_value(),3)
    Un_voxel.append(every)
np.array(Un_voxel)

array([ 61.152,  47.562,  33.949,  26.174,  25.007,  24.214,  18.259,
         9.912,   4.439,   3.194,   5.084,   8.617,  13.432,  19.914,
        25.296,  26.07 ,  24.15 ,  23.48 ,  25.042,  28.029,  62.145,
        56.334,  45.394,  36.214,  32.288,  29.444,  22.475,  13.489,
         7.251,   5.293,   7.144,  11.7  ,  18.345,  26.527,  32.36 ,
        32.188,  29.034,  27.762,  29.117,  31.306,  53.261,  57.048,
        53.198,  46.648,  41.72 ,  36.391,  27.367,  17.527,  11.187,
         9.35 ,  11.787,  17.641,  25.655,  34.268,  39.285,  37.734,
        33.492,  31.532,  31.912,  32.018,  38.024,  48.852,  53.721,
        53.761,  51.439,  45.158,  33.662,  22.23 ,  16.219,  16.006,
        20.22 ,  27.049,  34.731,  42.043,  45.545,  42.627,  37.184,
        33.943,  32.399,  29.716,  22.311,  34.944,  46.226,  54.323,
        58.158,  54.177,  41.722,  51.165,  46.585,  39.459,  34.292,
        30.289,  25.033,  10.63 ,  20.643,  59.101,  60.799,  49.345,
        26.022,  19.

In [33]:
print('Are there voxel values more than 72?')
max(Un_voxel) > 72

Are there voxel values more than 72?


True

In [34]:
print('how many percentage of voxel values are more than 72?')

print(round(sum((np.array(Un_voxel) >72)*1)/len(Un_voxel),2)*100,'%')

how many percentage of voxel values are more than 72?
21.0 %


**Left Femur Head**

In [35]:
# max dose to be voxel 50
Left_voxel = []
every = 0
for i in Left_index:
    every = round(voxels[i].solution_value(),2)
    Left_voxel.append(every)
print('left femur head voxel values')
np.array(Left_voxel)

left femur head voxel values


array([40.  , 32.46, 51.36, 39.08, 28.78, 20.34, 53.57, 37.34, 23.9 ,
       14.26, 56.28, 35.65, 19.21,  9.18, 58.43, 35.06, 16.86,  6.76,
       58.69, 36.09, 18.5 ,  8.23, 57.16, 38.13, 23.53, 13.69, 40.  ,
       29.29])

In [36]:
zl1 = np.array([[x.solution_value() for x in z_l_1]])
print('distances to 79.2 if voxel value is above 50\n',zl1)

distances to 79.2 if voxel value is above 50
 [[0.         0.         1.35684998 0.         0.         0.
  3.57373376 0.         0.         0.         6.28005042 0.
  0.         0.         8.42983812 0.         0.         0.
  8.69383655 0.         0.         0.         7.15625869 0.
  0.         0.         0.         0.        ]]


In [37]:
print('are there values more than 50?')
print(max(Left_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Left_voxel) >50)*1)/len(Left_voxel),2)*100,'%')

are there values more than 50?
True
how many percentage of voxel values are more than 50?
21.0 %


In [38]:
# at most 15% of the left femur head should receive > 40
y_left_value = np.array([t.solution_value() for t in y_left])
print('voxel that are larger that 40 are indicated by 1 \n',y_left_value)

voxel that are larger that 40 are indicated by 1 
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0.]


In [39]:
test_left_femur = np.array(list(map(lambda x: 1 if x >40.00 else 0,Left_voxel)))
print(round(sum(test_left_femur)/len(test_left_femur),4), ' of the left femur voxels are above 40')

0.2143  of the left femur voxels are above 40


In [40]:
print('voxel that are larger that 40 are indicated by 1 \n',test_left_femur)

voxel that are larger that 40 are indicated by 1 
 [0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0]


In [41]:
test_left_femur == y_left_value

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [42]:
print('the number of constraints we have to set loose to be above 40',z_y_left.solution_value())

the number of constraints we have to set loose to be above 40 2.0


**Right Femur Head**

In [43]:
# max dose to a voxel: 50 
Right_voxel = []
every = 0
for i in Right_index:
    every = round(voxels[i].solution_value(),2)
    Right_voxel.append(every)
print("right femur head voxel values")
np.array(Right_voxel)

right femur head voxel values


array([20.64, 33.62,  4.08, 10.03, 20.73, 36.68,  1.27,  4.12, 11.33,
       25.85,  0.37,  1.74,  6.67, 19.48,  0.3 ,  1.59,  6.58, 20.02,
        0.99,  3.67, 11.35, 28.19,  3.48,  9.54, 21.94, 42.74, 21.21,
       38.1 ])

In [44]:
# slack variable needed to fullfill the constraints
zrl1 = np.array([[x.solution_value() for x in z_rl_1]])
print('distances to 79.2 if voxel value is above 50\n',zrl1)

distances to 79.2 if voxel value is above 50
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]


In [45]:
# check if there's any right voxel value that is bigger than 50
print('are there values more than 50?')
print(max(Right_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Right_voxel) >50)*1)/len(Right_voxel),2)*100,'%')

are there values more than 50?
False
how many percentage of voxel values are more than 50?
0.0 %


In [46]:
# at most 15% of the right  femur head should receive > 40
y_right_value = np.array([t.solution_value() for t in y_right])
print('voxel that are larger that 40 are indicated by 1 \n',y_right_value)

voxel that are larger that 40 are indicated by 1 
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0.]


In [47]:
test_right = np.array(list(map(lambda x: 1 if x > 40.00 else 0,Right_voxel)))
print(round(sum(test_right)/len(test_right),4), ' of the right femur voxels are above 40')

0.0357  of the right femur voxels are above 40


In [48]:
print('voxel that are larger that 40 are indicated by 1 \n',test_right)

voxel that are larger that 40 are indicated by 1 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]


In [49]:
test_right == y_right_value

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [50]:
print('the number of constraints we have to set loose to be above 40',z_y_right.solution_value())

the number of constraints we have to set loose to be above 40 0.0
